# Comparaison des différentes transformées de Fourier.

In [ ]:
from __future__ import division
import holoviews as hv
hv.extension('bokeh')
import param
import holoviews as hv,panel as pn,param
from holoviews.streams import Pipe
import time
import pandas as pd
import panel as pn
from panel.pane import LaTeX
from scipy.io.wavfile import read
from IPython.display import Audio
import requests
from io import BytesIO
from PIL import Image
import shutil
from urllib.request import urlopen
import io
import scipy.io as sio

import numpy as np
import scipy as scp
import pylab as pyl
from matplotlib import cm
import matplotlib.pyplot as plt
from scipy import fftpack
from matplotlib.pyplot import imshow as imageplot
from mpl_toolkits.mplot3d import Axes3D
import wave
import warnings
warnings.filterwarnings('ignore')

# Lien entre les coefficients de Fourier d'une fonction périodique et la transformée de Fourier discrète.

Les programmes suivants vous permettent de comparer les coefficients de Fourier calculés de manière analytique de fonctions $2\pi$-périodiques $f$ et les coefficients de la transformée de Fourier discrète calculée sur un vecteur qui est une version échantillonnée de la fonction $f$. Vous pouves modifier le nombre $2N+1$ de points sur lequel est calculé la transformée discrète. Les coefficients de Fourier sont eux affichés entre $-N$ et $N$. Comme il est démontré dans la feuille 5 on a convergence en un certain sens des coefficients de la transformée de Fourier discrète de la version échantillonnée de $f$ vers les coefficients de Fourier $c_n(f)$ calculés de manière analytique, cest-à-dire via la formule que vous connaisez bien. 

## Exemple 1 : La fonction carré.

In [ ]:
def f1(x):
    y=np.mod(x+np.pi,2*np.pi)-np.pi
    return y**2

In [ ]:
def AfficheFourier1(N):
    Coefs=np.zeros(2*N+1)
    ind=np.arange(-N,N)
    for k in ind:
        if np.abs(k)>0:
            Coefs[k+N]=2*((-1)**(np.abs(k))/(k**2))
        else:
            Coefs[k+N]=np.pi**2/3
    C=hv.Points(Coefs,label='Coefficients de Fourier analytiques').opts(width=800)
    return C

In [ ]:
N=20
t=np.linspace(0,2*np.pi,2*N+1)
f=f1(t)
hv.Curve((t,f),label='Fonction carré périodisée')

In [ ]:
FT=np.real(fftpack.fft(f))/(2*N+1)
FT2=fftpack.fftshift(FT)
C=AfficheFourier1(N)
hv.Points(FT2,label='Coefficients de TF Discrète')*C

## Exemple 2, une fonction indicatrice.

In [ ]:
def f2(x,a):
    y=np.mod(x,2*np.pi)
    if y<a:
        res=1
    else:
        res=0
    return res
vf2=np.vectorize(f2)

In [ ]:
def AfficheFourier2(N,a):
    Coefs=np.zeros(2*N+1)
    ind=np.arange(-N,N)
    for k in ind:
        if np.abs(k)>0:
            Coefs[k+N]=np.abs((np.exp(-1j*k*a)-1)/k)
        else:
            Coefs[k+N]=2*np.pi/a
    C=hv.Points(Coefs,label='Coefficients de Fourier analytiques').opts(width=800)
    return C

In [ ]:
N=1000
a=0.3
eps=0.0000001
t=np.linspace(0,2*np.pi-eps,2*N+1)
f=vf2(t,a)
hv.Curve((t,f),label='Fonction indicatrice')


In [ ]:
FT=np.abs(fftpack.fft(f))/(2*N+1)
FT2=fftpack.fftshift(FT)*2*np.pi
C=AfficheFourier2(N,a)
hv.Points(FT2,label='Coefficients de TF Discrète')*C.opts(ylim=(0,a))

## Exemple 3, la fonction rampe.

In [ ]:
def f3(x):
    y=np.mod(x,2*np.pi)
    return y
vf3=np.vectorize(f3)

In [ ]:
def AfficheFourier3(N):
    Coefs=np.zeros(2*N+1)
    ind=np.arange(-N,N)
    for k in ind:
        if np.abs(k)>0:
            Coefs[k+N]=np.abs(1/k)
        else:
            Coefs[k+N]=np.pi
    C=hv.Points(Coefs,label='Coefficients de Fourier analytiques').opts(width=800)
    return C

In [ ]:
N=10
eps=0.0000001
t=np.linspace(0,2*np.pi-eps,2*N+1)
f=vf3(t)
hv.Curve((t,f),label='Fonction rampe')

In [ ]:
FT=np.abs(fftpack.fft(f))/(2*N+1)
FT2=fftpack.fftshift(FT)
C=AfficheFourier3(N)
hv.Points(FT2,label='Coefficients de TF Discrète')*C

# Lien entre la transformée de Fourier sur et la Transformée de Fourier discrète.

## Exemple 1, une fonction indicatrice.

L'objet de cet exemple est de comparer la transformée de Fourier discrète et la transformée de Fourier continue d'une fonction. Si on considère la fonction $f$ qui est l'indicatrice de l'intervalle $[0,1]$, on sait que sa transformée de Fourier s'exprime :
\begin{equation}
\hat f(\omega)=\frac{e^{-i\omega}-1}{-i\omega}
\end{equation}
On peut comparer (en module) cette formule avec la Transformée de Fourier discrète du vecteur obtenu par discrétisation de cette fonction. Vous pouvez faire varier la valeur de $N$ et voir que plus on discrétise finement plus la convergence est bonne.

In [ ]:
def tfind(omega):
    a=1
    return (np.exp(-1j*omega*a)-1)*1j/omega

vtfind=np.vectorize(tfind)

In [ ]:
N=50
a=1
eps=0.0000001
t=np.linspace(0,2*np.pi-eps,2*N+1)
f=vf2(t,a)
hv.Curve((t,f),label='fonction indicatrice').opts(width=800)

In [ ]:

FT=np.abs(fftpack.fft(f))/(2*N+1)
FT2=fftpack.fftshift(FT)*2*np.pi
t=np.linspace(-N,N,2*N+1)
tf=np.abs(vtfind(t))
C3=hv.Curve(tf,label='Transformée de Fourier continue')
C3*hv.Points(FT2,label='Coefficients de TF Discrète').opts(ylim=(0,a),color='red').opts(width=800)


## Exemple 2, la gaussienne.

In [ ]:
def gaus(t):
    return np.exp(-t**2/2)
vgaus=np.vectorize(gaus)

In [ ]:
N=81
A=np.sqrt(N*np.pi/2)
t=np.linspace(-A,A,N)
fg=gaus(t)
hv.Curve((t,fg),label='Gaussienne').opts(width=800)


On rappelle que la transformée de Fourier de la gaussienne est la gaussienne à facteur multiplciatif près, comme il l'est démontré en TD. Plus $N$ est grand, plus on est proche de la vraie transformée de Fourier. De fait les points rouges ne sont pas exactement sur la courbe bleu.

In [ ]:
Cg1=hv.Curve((t,fg/np.sqrt(2)),label='Transformée analytique de Fourier de la Gaussienne').opts(width=800)
FT=np.abs(fftpack.fft(fg))/np.sqrt((2*N+1))
FT2=fftpack.fftshift(FT)
Cg2=hv.Points((t,np.abs(FT2)),label='TF discrète de la gaussienne échantillonnée').opts(width=800,color='red')
Cg1*Cg2

## Exemple 3 : La gaussienne avec une variance non unitaire.

Nous avons vu en cours que la dilatation en espace induisait une contraction dans le domaine de Fourier. On peut bien entendu l'observer numériquement en faisant varier le paramètre de variance dans la gaussienne. Vous pouvez faire quelques tests en faisant varier sigma dans le programme ci dessous. Vous oberseveez cependant une différence notoire entre la transformée discrète et la trasnformée continue analytique si sigma est trop grand... ceci est dû au fait que dès qu'on travaille en discret, on périodise moralement le signal... ce qui fait que si la variance est trop grande, les différentes gaussiennes issues de la périodisation se téléscopent... nous aurons un problème analogue quand nous traiterons de l'écahntillonnage des signaux.

In [ ]:
sigma=0.1
def gaus2(t):
    return np.exp(-sigma*t**2/2)
def gaus3(t):
    return np.exp(-t**2/(sigma*2))
N=81
A=np.sqrt(N*np.pi/2)
t=np.linspace(-A,A,N)
fg2=gaus2(t)
fg3=gaus3(t)
Cg1=hv.Curve((t,fg3/np.sqrt(2)),label='Transformée analytique de Fourier de la Gaussienne').opts(width=800)
FT=np.abs(fftpack.fft(fg2))/np.sqrt((2*N+1)/sigma)
FT2=fftpack.fftshift(FT)
Cg2=hv.Points((t,np.abs(FT2)),label='TF discrète de la gaussienne échantillonnée').opts(width=800,color='red')
Cg1*Cg2